In [1]:
import numpy as pn
import pandas as pd

In [2]:
room_stat = pd.read_csv('roomstatinfo.csv')
room_stat.head()

,CompanyName,记录数,companyId,depId,duration,endTime,name,roomId,startTime,userCount
0,长寿村,1,130261,154963,16099,2017/3/5 8:25:28,NaN,280025,2017/3/5 3:57:09,4
1,金日鸿雁,1,126765,151915,1,2017/3/5 5:33:38,NaN,270149,2017/3/5 5:33:37,0
2,金日鸿雁,1,126765,151915,43,2017/3/5 5:34:29,NaN,270149,2017/3/5 5:33:46,0
3,福建源和康商贸有限公司,1,132015,156785,8044,2017/3/5 8:09:54,NaN,284749,2017/3/5 5:55:50,4
4,伊美时健康生活馆,1,132379,157155,36408,2017/3/5 16:52:11,周雪萍,285163,2017/3/5 6:45:23,19


In [3]:
room_stat["startTime"] = pd.to_datetime(room_stat['startTime'])
group_room_stat = room_stat.set_index('startTime').groupby([ "CompanyName"]).resample("W").size().to_frame(name = 'count').reset_index()
#group_room_stat

In [4]:
chair = pd.read_csv('roomlogchair.csv')
#chair.head()

In [5]:
chair = pd.read_csv('roomlogchair.csv')
chair["EnterTime"] = pd.to_datetime(chair["EnterTime"])
group_chair = chair.set_index('EnterTime').groupby(["CompanyName"]).resample("W").size().to_frame(name = 'count').reset_index()
#group_chair

In [6]:
user = pd.read_csv('userlog.csv')
user["LogTime"] = pd.to_datetime(user['LogTime'])
group_user = user.set_index('LogTime').groupby(["CompanyName"]).resample("W").size().to_frame(name = 'count').reset_index()
#group_user

In [7]:
group_stat_user_merge = pd.merge(group_room_stat,group_user,how = 'left',left_on = ['CompanyName','startTime'],right_on = ['CompanyName','LogTime'])
#group_stat_user_merge

In [8]:
group_merge_all = pd.merge(group_stat_user_merge,group_chair,how = 'left',left_on = ['CompanyName','startTime'],right_on = ['CompanyName','EnterTime'])
#group_merge_all

In [9]:
group_merge_all.rename(index=str, columns={"count_x": "meeting_count", "count_y": "key_user_count","count": "chair_count"},inplace=True)
#group_merge_all

In [10]:
group_merge_all.rename(index=str, columns={"meeting_count": "会议次数", "key_user_count": "操作员登陆次数","chair_count": "主席登陆次数"},inplace=True)
#group_merge_all

In [11]:
group_merge_all['startTime'] = group_merge_all['startTime'].apply(str)
group_merge_all['LogTime'] = group_merge_all['LogTime'].apply(str)
group_merge_all['EnterTime'] = group_merge_all['EnterTime'].apply(str)
type(group_merge_all['EnterTime'][10])

str

In [12]:
group_merge_all['startTime'] = group_merge_all['startTime'].str.replace(' 00:00:00','')
group_merge_all['LogTime'] = group_merge_all['LogTime'].str.replace(' 00:00:00','')
group_merge_all['EnterTime'] = group_merge_all['EnterTime'].str.replace(' 00:00:00','')
group_merge_all['EnterTime'] = group_merge_all['EnterTime'].str.replace('NaT','0')
group_merge_all['LogTime'] = group_merge_all['LogTime'].str.replace('NaT','0')
#group_merge_all

In [13]:
group_merge_all['主席登陆次数'] = group_merge_all['主席登陆次数'].fillna(0)
group_merge_all['操作员登陆次数'] = group_merge_all['操作员登陆次数'].fillna(0)
#group_merge_all

In [14]:
group_merge_all.corr()

,会议次数,操作员登陆次数,主席登陆次数
会议次数,1.000000,0.703298,0.755498
操作员登陆次数,0.703298,1.000000,0.485271
主席登陆次数,0.755498,0.485271,1.000000


In [15]:
import matplotlib as mpl
mpl.rcParams['font.serif'] = ['SimHei']
import seaborn as sns
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})
%matplotlib inline

#sns.pairplot(group_merge_all)